In [2]:
import tushare as ts
from sklearn.model_selection import train_test_split
import datetime
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score



In [3]:
ts.set_token('4f2c81a89e1a43100569f800f658a0a26f55ba57c93908691c17b681')
pro = ts.pro_api()

交易所代码 CFFEX-中金所 DCE-大商所 CZCE-郑商所 SHFE-上期所 INE-上海国际能源交易中心 GFEX-广州期货交易所

In [17]:
today = datetime.date.today()

# 计算50天前的日期
delta = datetime.timedelta(days=50)
target_date = today - delta

# 创建时间对象
time1 = datetime.time(hour=15, minute=0, second=0)
time2 = datetime.time(hour=9, minute=0, second=0)

# 创建完整的日期时间对象
datetime1 = datetime.datetime.combine(today, time1)
datetime2 = datetime.datetime.combine(target_date, time2)
datetime1_str = datetime1.strftime('%Y-%m-%d %H:%M:%S')
datetime2_str = datetime2.strftime('%Y-%m-%d %H:%M:%S')

In [18]:
print(datetime1)
print(datetime2)

2023-04-20 15:00:00
2023-03-01 09:00:00


In [19]:
df = ts.pro_bar(ts_code='OI2309.ZCE',
                    asset='FT',
                    freq='15min', 
                    start_date= datetime2_str,
                    end_date= datetime1_str)    

抱歉，您每天最多访问该接口5次，权限的具体详情访问：https://tushare.pro/document/1?doc_id=108。
抱歉，您每分钟最多访问该接口1次，权限的具体详情访问：https://tushare.pro/document/1?doc_id=108。
抱歉，您每分钟最多访问该接口1次，权限的具体详情访问：https://tushare.pro/document/1?doc_id=108。


OSError: ERROR.

In [7]:
df['change1'] = df['close'] - df['close'].shift(1)
df['change1'].iloc[0:2] = 0

df.insert(0, 'number', range(1, len(df)+1))

C:\Users\DuYih\AppData\Local\Temp\ipykernel_26684\261944342.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['change1'].iloc[0:2] = 0


In [8]:
df.to_csv('futures_min.csv', index=False)

In [9]:
# 读入数据
# df = pd.read_csv('futures.csv')

# 构建特征矩阵
features = ['change1', 'open', 'close', 'oi', 'vol']
X = np.array(df[features])
y = np.array(df['change1'] > 0)

# 划分训练集和测试集
train_size = int(0.7 * len(df))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# 建立随机森林模型
rfc = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
rfc.fit(X_train, y_train)

# 预测测试集
y_pred = rfc.predict(X_test)

# 评估模型准确度
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 1.0


In [10]:
# 假设你已经得到了一个新的数据点，保存在一个名为new_data的DataFrame中
new_data = pd.DataFrame({'change1': [84], 
                         'open': [4777], 
                         'close': [4776], 
                         'oi': [53], 
                         'vol': [824]})

# 对新数据进行预测
y_pred = rfc.predict_proba(new_data)

# 输出涨跌概率
print('下跌概率:', y_pred[:, 0])
print('上涨概率:', y_pred[:, 1])

下跌概率: [0.115]
上涨概率: [0.885]


c:\Users\DuYih\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [11]:
# 加载数据
# data = pd.read_csv('futures.csv')
data=df

# 构建特征矩阵
features = ['open', 'close', 'high', 'low']
X = []
for i in range(3, len(data)):
    x_i = data.loc[i-3:i-1, features].values.reshape(-1)
    X.append(x_i)
X = np.array(X)

# 标记目标变量
y = np.where(data['change1'][3:] > 0, 1, 0)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 训练模型
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(X_train, y_train)

# 评估模型
y_pred = rfc.predict(X_test)
print('准确率：', accuracy_score(y_test, y_pred))
print('精确率：', precision_score(y_test, y_pred))
print('召回率：', recall_score(y_test, y_pred))

准确率： 0.8125
精确率： 0.8333333333333334
召回率： 0.8333333333333334


In [12]:
# 构造新数据
new_data = data.loc[len(data)-3:len(data)-1, features].values.reshape(-1, 12)

# 预测涨跌概率
y_pred = rfc.predict_proba(new_data)

# 输出涨跌概率
print('下跌概率:', y_pred[0, 0])
print('上涨概率:', y_pred[0, 1])

下跌概率: 0.57
上涨概率: 0.43
